# Snorkel

## Introduction

The goal of this lab is to introduce students to the [Snorkel](http://www.snorkel.org) tool and the possibilities of programmatic label generation using the weak-supervised learning paradigm.

In order to use weakly supervised learning to generate labels, it is necessary to create three datasets:

- **train set**: which does not have any labels
- **validation set**: used for hyperparameter optimization, has labels
- **test set**: used only for final model evaluation, has labels

## Labeling functions

The first step will be to load the dataset and split it into a train set and a test set. Since in our set all SMS have a label, we will simulate a weakly supervised learning problem by randomly removing 80% of the labels. Additionally, Snorkel requires numeric labels, so we need to recode the values.

In [5]:
!head data/smsspamcollection.csv

'head' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
import pandas as pd
import numpy as np

pd.set_option('max_colwidth', 600)

SPAM = 1
HAM = 0
ABSTAIN = -1

df = pd.read_csv('./data/smsspamcollection.csv', sep='\t', header=None, names=['old_label', 'text'])

df['label'] = df.old_label.apply(lambda x: SPAM if x == 'spam' else HAM)

df.loc[df.sample(frac=0.8).index, 'label'] = ABSTAIN
df.drop(columns=['old_label'], inplace=True)

df.head()

,text,label
0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",-1
1,Ok lar... Joking wif u oni...,-1
2,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,-1
3,U dun say so early hor... U c already then say...,-1
4,"Nah I don't think he goes to usf, he lives around here though",0


In [7]:
abstain_idx = df.label == ABSTAIN

df_train = df[abstain_idx]
df_test = df[~abstain_idx]

### Simple keyword search

As a first example, we will use a search for the words "check" and "free" in SMS content

In [171]:
from snorkel.labeling import labeling_function

@labeling_function()
def check(sms):
    return SPAM if "check" in sms.text.lower() else ABSTAIN

@labeling_function()
def free(sms):
    return SPAM if "free" in sms.text.lower() else ABSTAIN

The next step is to apply the labeling functions to the train set.

In [172]:
from snorkel.labeling import PandasLFApplier

lfs = [check, free]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 4458/4458 [00:00<00:00, 55467.67it/s]


The result of applying the set of labeling functions to the train set is a matrix of size $m \times n$, where $m$ is the number of examples and $n$ is the number of labeling functions. The matrix contains the result of applying each function to each example.

In [10]:
L_train

array([[-1, -1],
       [-1, -1],
       [-1,  1],
       ...,
       [-1, -1],
       [-1,  1],
       [-1, -1]])

In [11]:
df_train.iloc[1,:]

text     Ok lar... Joking wif u oni...
label                               -1
Name: 1, dtype: object

The simplest way to analyze this is to determine the coverage of labeling functions (i.e., the percentage of cases for which the function returned a result other than `ABSTAIN'.

In [12]:
coverage_check, coverage_free = (L_train != ABSTAIN).mean(axis=0)

print(f"Coverage for check(): {coverage_check * 100:.1f}%")
print(f"Coverage for free(): {coverage_free * 100:.1f}%")

Coverage for check(): 1.1%
Coverage for free(): 4.8%


Fortunately, Snorkel offers additional tools that allow for deeper analysis of the result of labeling functions.

In [13]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.0,0.0
free,1,[1],0.048228,0.0,0.0


The meaning of each column is as follows:
- `Polarity`: the set of labels returned by the function
- `Coverage`: the percentage of examples for which the function returns a value other than `ABSTAIN`
- Overlaps: the percentage of examples for which at least one other labeling function returned a value
- Conflicts: the percentage of examples for which at least one other labeling function returned a different value

If the train set contained labels, the method would also return:
- `Correct`: the number of correct labels
- `Incorrect`: number of incorrect labels
- `Empirical Accuracy`: the percentage of correct labels

Let's check the examples labeled by the `free()` function as spam

In [173]:
df_train.iloc[L_train[:,1] == SPAM].sample(frac=0.1)

,text,label
1759,Sorry i'm not free...,-1
1518,Our brand new mobile music service is now live. The free music player will arrive shortly. Just install on your phone to browse content from the top artists.,-1
2583,"3 FREE TAROT TEXTS! Find out about your love life now! TRY 3 FOR FREE! Text CHANCE to 85555 16 only! After 3 Free, Msgs £1.50 each",-1
5131,That sucks. I'll go over so u can do my hair. You'll do it free right?,-1
4221,U free on sat rite? U wan 2 watch infernal affairs wif me n darren n mayb xy?,-1
5196,"Spook up your mob with a Halloween collection of a logo & pic message plus a free eerie tone, txt CARD SPOOK to 8007 zed 08701417012150p per logo/pic",-1
4077,87077: Kick off a new season with 2wks FREE goals & news to ur mobile! Txt ur club name to 87077 eg VILLA to 87077,-1
3763,FREE for 1st week! No1 Nokia tone 4 ur mob every week just txt NOKIA to 8007 Get txting and tell ur mates www.getzed.co.uk POBox 36504 W45WQ norm150p/tone 16+,-1
492,"Congrats! 1 year special cinema pass for 2 is yours. call 09061209465 now! C Suprman V, Matrix3, StarWars3, etc all 4 FREE! bx420-ip4-5we. 150pm. Dont miss out!",-1
2311,You have won a Nokia 7250i. This is what you get when you win our FREE auction. To take part send Nokia to 86021 now. HG/Suite342/2Lands Row/W1JHL 16+,-1


It seems that the phrase "call now" is also a good indicator for spam. So let's add one more labeling function.

In [174]:
@labeling_function()
def call_now(sms):
    return SPAM if "call now" in sms.text.lower() else ABSTAIN

lfs = [check, free, call_now]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 4458/4458 [00:00<00:00, 40942.08it/s]


In [175]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.000000,0.0
free,1,[1],0.048228,0.000897,0.0
call_now,2,[1],0.003140,0.000897,0.0


Let's see which examples were labeled as spam by the `call_now()` function but omitted by `free()`.

In [176]:
from snorkel.analysis import get_label_buckets

buckets = get_label_buckets(L_train[:, 1], L_train[:, 2])
buckets

{(-1, -1): array([   0,    1,    3, ..., 4454, 4455, 4457]),
 (1,
  -1): array([   2,    6,    9,   57,   68,   72,  104,  108,  125,  130,  131,
         137,  169,  187,  203,  226,  241,  270,  278,  292,  295,  307,
         322,  353,  378,  382,  405,  445,  458,  473,  516,  529,  557,
         616,  621,  631,  639,  712,  753,  797,  800,  852,  876,  908,
         930,  955,  972,  976,  978, 1090, 1105, 1136, 1153, 1164, 1193,
        1203, 1204, 1214, 1229, 1236, 1261, 1290, 1302, 1316, 1354, 1364,
        1401, 1415, 1423, 1433, 1436, 1445, 1526, 1554, 1580, 1584, 1602,
        1611, 1619, 1667, 1670, 1697, 1709, 1738, 1761, 1792, 1825, 1845,
        1854, 1862, 1899, 1905, 1908, 1916, 1998, 2028, 2063, 2080, 2088,
        2122, 2137, 2171, 2262, 2266, 2296, 2303, 2317, 2320, 2335, 2395,
        2404, 2415, 2420, 2425, 2512, 2516, 2558, 2650, 2663, 2666, 2677,
        2700, 2716, 2743, 2746, 2757, 2801, 2810, 2827, 2873, 2878, 2940,
        2954, 2968, 2985, 3011, 3018, 30

In [177]:
df_train.iloc[buckets[(ABSTAIN, SPAM)]]

,text,label
1366,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870..k",-1
1502,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national rate call",-1
2170,"Shop till u Drop, IS IT YOU, either 10K, 5K, £500 Cash or £100 Travel voucher, Call now, 09064011000. NTT PO Box CR01327BT fixedline Cost 150ppm mobile vary",-1
2850,"YOUR CHANCE TO BE ON A REALITY FANTASY SHOW call now = 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national = rate call",-1
3167,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870..k",-1
3893,URGENT This is our 2nd attempt to contact U. Your £900 prize from YESTERDAY is still awaiting collection. To claim CALL NOW 09061702893. ACL03530150PM,-1
4073,Loans for any purpose even if you have Bad Credit! Tenants Welcome. Call NoWorriesLoans.com on 08717111821,-1
4283,U can call now...,-1
4371,Do you want a new Video handset? 750 any time any network mins? UNLIMITED TEXT? Camcorder? Reply or Call now 08000930705 for del Sat AM,-1
5481,Shall call now dear having food,-1


In [178]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.000000,0.0
free,1,[1],0.048228,0.000897,0.0
call_now,2,[1],0.003140,0.000897,0.0


#### assignment

Write a labeling function that marks as spam all messages containing the word "HOT" written in capitals.

In [179]:
@labeling_function()
def hot(sms):
    return SPAM if "HOT" in sms.text else ABSTAIN

### Searching based on a regular expression

Another type of labeling function is one that uses regexp to find specific expressions.

In [181]:
import re

@labeling_function()
def regex_I_am_free(sms):
    if re.search(r"I\s.*free", sms.text, flags=re.I):
        return HAM
    elif re.search(r"free", sms.text, flags=re.I):
        return SPAM
    else:
        return ABSTAIN

lfs = [check, free, call_now, regex_I_am_free]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 23831.55it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.000000,0.000000
free,1,[1],0.048228,0.048228,0.005159
call_now,2,[1],0.003140,0.000897,0.000000
regex_I_am_free,3,"[0, 1]",0.048228,0.048228,0.005159


Let's compare examples that the `free()` function labels as spam and the `regex_I_am_free()` function considers valid.

In [182]:
buckets = get_label_buckets(L_train[:, 1], L_train[:, 3])
df_train.iloc[buckets[(SPAM, HAM)]].sample(10)

,text,label
2370,"A Boy loved a gal. He propsd bt she didnt mind. He gv lv lttrs, Bt her frnds threw thm. Again d boy decided 2 aproach d gal , dt time a truck was speeding towards d gal. Wn it was about 2 hit d girl,d boy ran like hell n saved her. She asked 'hw cn u run so fast?' D boy replied ""Boost is d secret of my energy"" n instantly d girl shouted ""our energy"" n Thy lived happily 2gthr drinking boost evrydy Moral of d story:- I hv free msgs:D;): gud ni8",-1
948,Hey i booked the kb on sat already... what other lessons are we going for ah? Keep your sat night free we need to meet and confirm our lodging,-1
3692,"Sir, i am waiting for your call, once free please call me.",-1
3595,Do you want a New Nokia 3510i Colour Phone Delivered Tomorrow? With 200 FREE minutes to any mobile + 100 FREE text + FREE camcorder Reply or Call 08000930705,-1
5104,"A Boy loved a gal. He propsd bt she didnt mind. He gv lv lttrs, Bt her frnds threw thm. Again d boy decided 2 aproach d gal , dt time a truck was speeding towards d gal. Wn it was about 2 hit d girl,d boy ran like hell n saved her. She asked 'hw cn u run so fast?' D boy replied ""Boost is d secret of my energy"" n instantly d girl shouted ""our energy"" n Thy lived happily 2gthr drinking boost evrydy Moral of d story:- I hv free msgs:D;): gud ni8",-1
4386,Do you want a New Nokia 3510i Colour Phone Delivered Tomorrow? With 200 FREE minutes to any mobile + 100 FREE text + FREE camcorder Reply or Call 8000930705,-1
3778,"Mila, age23, blonde, new in UK. I look sex with UK guys. if u like fun with me. Text MTALK to 69866.18 . 30pp/txt 1st 5free. £1.50 increments. Help08718728876",-1
4587,"Mila, age23, blonde, new in UK. I look sex with UK guys. if u like fun with me. Text MTALK to 69866.18 . 30pp/txt 1st 5free. £1.50 increments. Help08718728876",-1
4949,"Hi this is Amy, we will be sending you a free phone number in a couple of days, which will give you an access to all the adult parties...",-1
5570,The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free,-1


#### assignment

Write a labeling function that will mark as spam all messages containing any amounts specified with a currency symbol ($99, £1.50)

In [186]:
@labeling_function()
def contains_money(sms):
    return SPAM if re.search(r"\$", sms.text) or re.search(r"£", sms.text) else ABSTAIN


In [187]:
lfs = [check, free, contains_money]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

buckets = get_label_buckets(L_train[:, 1], L_train[:, 2])

df_train.iloc[buckets[(ABSTAIN, SPAM)]]

100%|██████████| 4458/4458 [00:00<00:00, 30121.38it/s]


,text,label
34,Thanks for your subscription to Ringtone UK your mobile will be charged £5/month Please confirm by replying YES or NO. If you reply NO you will not be charged,-1
60,Your gonna have to pick up a $1 burger for yourself on your way home. I can't even move. Pain is killing me.,-1
65,"As a valued customer, I am pleased to advise you that following recent review of your Mob No. you are awarded with a £1500 Bonus Prize, call 09066364589",-1
93,Please call our customer service representative on 0800 169 6031 between 10am-9pm as you have WON a guaranteed £1000 cash or £5000 prize!,-1
114,GENT! We are trying to contact you. Last weekends draw shows that you won a £1000 prize GUARANTEED. Call 09064012160. Claim Code K52. Valid 12hrs only. 150ppm,-1
...,...,...
5260,If anyone calls for a treadmill say you'll buy it. Make sure its working. I found an ad on Craigslist selling for $ &lt;#&gt; .,-1
5292,Urgent! Please call 09061213237 from landline. £5000 cash or a luxury 4* Canary Islands Holiday await collection. T&Cs SAE PO Box 177. M227XY. 150ppm. 16+,-1
5314,Get the official ENGLAND poly ringtone or colour flag on yer mobile for tonights game! Text TONE or FLAG to 84199. Optout txt ENG STOP Box39822 W111WX £1.50,-1
5482,URGENT We are trying to contact you Last weekends draw shows u have won a £1000 prize GUARANTEED Call 09064017295 Claim code K52 Valid 12hrs 150p pm,-1


### Searching based on heuristics

A simple heuristic to find spam is to assume that if more than 10% of the message text is written in capitals, there is a good chance it is spam.

In [24]:
@labeling_function()
def has_many_uppercase_words(sms):
    percentage_uppercase = sum([word.isupper() for word in sms.text.split()]) / len(sms.text.split())
    
    return SPAM if percentage_uppercase > 0.1 else ABSTAIN

lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 15587.79it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.002019,0.000000
free,1,[1],0.048228,0.048228,0.005159
call_now,2,[1],0.003140,0.002692,0.000000
regex_I_am_free,3,"[0, 1]",0.048228,0.048228,0.005159
has_many_uppercase_words,4,[1],0.184836,0.020188,0.001795


#### assignment

Write a labeling function that marks as valid those messages that are shorter than 10 words and do not contain any word written in capitals.

In [25]:
@labeling_function()
def short_and_no_uppercase(sms):
    pass

### Using an external statistical model

When labeling data, you can use external models whose response can be important information for deciding how to label an example. Snorkel has several built-in integrations in the form of the `Preprocessor` interface, in the example below we will use the `SpaCy` library to perform additional grammatical analysis of the text. However, you will need to download the English language model.

In [26]:
!python -m spacy download en_core_web_sm

                                              0.0/12.8 MB ? eta -:--:--
                                              0.0/12.8 MB 2.0 MB/s eta 0:00:07
                                              0.1/12.8 MB 1.2 MB/s eta 0:00:11
                                              0.2/12.8 MB 1.7 MB/s eta 0:00:08
     -                                        0.5/12.8 MB 2.4 MB/s eta 0:00:06
     --                                       0.7/12.8 MB 2.8 MB/s eta 0:00:05
     --                                       0.8/12.8 MB 3.2 MB/s eta 0:00:04
     ---                                      1.1/12.8 MB 3.4 MB/s eta 0:00:04
     ----                                     1.3/12.8 MB 3.5 MB/s eta 0:00:04
     ----                                     1.5/12.8 MB 3.7 MB/s eta 0:00:04
     -----                                    1.8/12.8 MB 4.0 MB/s eta 0:00:03
     ------                                   2.0/12.8 MB 3.9 MB/s eta 0:00:03
     -------                                  2.3/12.8 MB 4

In [27]:
!python -m spacy validate


⠙ Loading compatibility table...
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.5.2) =================
ℹ spaCy installation:
C:\Users\erykp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\spacy

NAME             SPACY            VERSION                            
en_core_web_md   >=3.5.0,<3.6.0   3.5.0   ✔
en_core_web_sm   >=3.5.0,<3.6.0   3.5.0   ✔



In [28]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [29]:
_text = """I don't England is a country that is part of the United Kingdom. 
It shares land borders with Wales to its west and Scotland to its north. 
The Irish Sea lies northwest of England and the Celtic Sea to the southwest. 
England is separated from continental Europe by the North Sea to the east and the 
English Channel to the south. The country covers five-eighths of the island of 
Great Britain, which lies in the North Atlantic, and includes over 100 smaller islands, 
such as the Isles of Scilly and the Isle of Wight."""

doc = nlp(_text)

for e in doc.ents:
    print(e.text, e.label_)

England GPE
the United Kingdom GPE
Wales ORG
Scotland GPE
The Irish Sea LOC
England GPE
the Celtic Sea LOC
England GPE
Europe LOC
the North Sea LOC
English LANGUAGE
five-eighths CARDINAL
Great Britain GPE
the North Atlantic LOC
over 100 CARDINAL
the Isles of Scilly GPE
the Isle of Wight GPE


In [30]:
from snorkel.preprocess.nlp import SpacyPreprocessor

spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

Assume that short text messages in which a reference to a specific person appears are not spam.

In [31]:
df_train.columns

Index(['text', 'label'], dtype='object')

In [32]:
@labeling_function(pre=[spacy])
def has_person(sms):
    if len(sms.doc) < 20 and any([ent.label_ == "PERSON" for ent in sms.doc.ents]):
        return HAM
    else:
        return ABSTAIN

In [33]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:28<00:00, 155.99it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.002916,0.000897
free,1,[1],0.048228,0.048228,0.006281
call_now,2,[1],0.003140,0.002692,0.000000
regex_I_am_free,3,"[0, 1]",0.048228,0.048228,0.006281
has_many_uppercase_words,4,[1],0.184836,0.025572,0.007178
has_person,5,[0],0.046209,0.007402,0.007402


Another example of pre-processing data for labeling would be determining the average word frequency of a document. Below we define a function that determines the average word frequency and we decorate it as an example of a pre-processor. When a text message is sent to the next labeling function, the pre-processor will populate the text message with the average word frequency and, based on that, the labeling function will make a decision (we assume that if the text message contains many rare words then it is spam).

In [35]:
from wordfreq import zipf_frequency
from snorkel.preprocess import preprocessor

@preprocessor(memoize=True)
def avg_word_freq(sms):
    sms.avg_word_freq = sum([zipf_frequency(word, 'en') for word in sms.text.split()]) / len(sms.text.split())
    
    return sms

In [36]:
@labeling_function(pre=[avg_word_freq])
def many_rare_words(sms):
    return ABSTAIN if sms.avg_word_freq >= 4 else SPAM

In [37]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person, many_rare_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:02<00:00, 1913.62it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.003365,0.000897
free,1,[1],0.048228,0.048228,0.006281
call_now,2,[1],0.003140,0.002692,0.000000
regex_I_am_free,3,"[0, 1]",0.048228,0.048228,0.006281
has_many_uppercase_words,4,[1],0.184836,0.045312,0.007178
has_person,5,[0],0.046209,0.014805,0.014805
many_rare_words,6,[1],0.064154,0.034545,0.008973


In [38]:
df_train.iloc[L_train[:,6] == SPAM].sample(frac=0.1)

,text,label
349,Fancy a shag? I do.Interested? sextextuk.com txt XXUK SUZY to 69876. Txts cost 1.50 per msg. TnCs on website. X,-1
2599,Okie... Thanx...,-1
3679,Promotion Number: 8714714 - UR awarded a City Break and could WIN a £200 Summer Shopping spree every WK. Txt STORE to 88039 . SkilGme. TsCs087147403231Winawk!Age16 £1.50perWKsub,-1
783,Beerage?,-1
5478,No probably &lt;#&gt; %.,-1
5526,PRIVATE! Your 2003 Account Statement for shows 800 un-redeemed S.I.M. points. Call 08718738001 Identifier Code: 49557 Expires 26/11/04,-1
3507,Nite...,-1
2074,"FreeMsg: Claim ur 250 SMS messages-Text OK to 84025 now!Use web2mobile 2 ur mates etc. Join Txt250.com for 1.50p/wk. T&C BOX139, LA32WU. 16 . Remove txtX or stop",-1
4153,What's nannys address?,-1
4169,Congrats! Nokia 3650 video camera phone is your Call 09066382422 Calls cost 150ppm Ave call 3mins vary from mobiles 16+ Close 300603 post BCM4284 Ldn WC1N3XX,-1


#### assignment

Write a labeling function that marks messages containing more than 3 adjectives as spam. Use the SpaCy library for pre-processing. 

__Hint__: the following example shows how to read the part-of-speech label for each token from the message being analyzed. For information on all token properties recognized by SpaCy, see [API documentation](https://spacy.io/api/token)

In [39]:
import spacy

nlp = spacy.load('en_core_web_sm')

sms = "Yetunde, i'm sorry but moji and i seem too busy to be able to go shopping."

for token in nlp(sms):
    print(f"{token.text:<10} {token.pos_:<10} {token.tag_:<10} {token.lemma_:<10}")

Yetunde    PROPN      NNP        Yetunde   
,          PUNCT      ,          ,         
i          PRON       PRP        I         
'm         AUX        VBP        be        
sorry      ADJ        JJ         sorry     
but        CCONJ      CC         but       
moji       ADJ        JJ         moji      
and        CCONJ      CC         and       
i          PRON       PRP        I         
seem       VERB       VBP        seem      
too        ADV        RB         too       
busy       ADJ        JJ         busy      
to         PART       TO         to        
be         AUX        VB         be        
able       ADJ        JJ         able      
to         PART       TO         to        
go         VERB       VB         go        
shopping   NOUN       NN         shopping  
.          PUNCT      .          .         


## Combining labeling functions into a single model

The goal of labeling functions is not to achieve individually large coverage. Labeling functions are inherently noisy and can make many individual errors. The true utility of labeling functions becomes apparent when multiple functions are combined to form a single model.

We will first build a simple model based on majority voting, and then build a more complex model. 

In [40]:
lfs = [check, free, call_now, regex_I_am_free, has_person, many_rare_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 1114/1114 [00:08<00:00, 127.45it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.001346,0.000897
free,1,[1],0.048228,0.048228,0.006281
call_now,2,[1],0.003140,0.001346,0.000000
regex_I_am_free,3,"[0, 1]",0.048228,0.048228,0.006281
has_person,4,[0],0.046209,0.010094,0.010094
many_rare_words,5,[1],0.064154,0.014805,0.008973


In [41]:
LFAnalysis(L=L_test, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.008079,0.001795,0.001795
free,1,[1],0.044883,0.044883,0.008977
call_now,2,[1],0.007181,0.001795,0.000000
regex_I_am_free,3,"[0, 1]",0.044883,0.044883,0.008977
has_person,4,[0],0.062837,0.010772,0.010772
many_rare_words,5,[1],0.060144,0.016158,0.008977


In [42]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)

In [43]:
preds_train

array([-1,  1,  1, ..., -1, -1, -1])

In [44]:
import numpy as np

labels, counts = np.unique(preds_train, return_counts=True)

for l, c in zip(labels, counts):
    print(f"LABEL: {l}, count: {c}")

LABEL: -1, count: 3825
LABEL: 0, count: 161
LABEL: 1, count: 472


In [45]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=42)

INFO:root:Computing O...
INFO:root:Estimating \mu...
 38%|███▊      | 190/500 [00:00<00:00, 1001.77epoch/s]INFO:root:[200 epochs]: TRAIN:[loss=0.003]
INFO:root:[300 epochs]: TRAIN:[loss=0.002]
100%|██████████| 500/500 [00:00<00:00, 1130.34epoch/s]
INFO:root:Finished Training


In [46]:
majority_acc = majority_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Majority voting accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Probabilistic model accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority voting accuracy: 50.9%
Probabilistic model accuracy: 51.3%


Unfortunately, some data points will not receive any label. It is necessary to filter out these points before sending the labeling result for further processing.

In [47]:
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.utils import preds_to_probs, probs_to_preds

preds_train, probs_train = label_model.predict(L=L_train, return_probs=True)

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(X=df_train, y=probs_train, L=L_train)
df_train.shape, df_train_filtered.shape

((4458, 2), (693, 2))

As you can see, we were able to quickly prepare labels for about 650 examples (recall that initially no example in the `df_train` set had labels).

The next step will use prepared labels as training data for the actual classifier. We will use simple [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), first pre-processing the input data. Since we are working with text, we will use the [word vector representation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) created based on 5-grams by `CountVectorizer`.

In [48]:
from snorkel.utils import probs_to_preds
from sklearn.feature_extraction.text import CountVectorizer

preds_train_filtered = probs_to_preds(probs=probs_train_filtered)

vectorizer = CountVectorizer(ngram_range=(1, 5))

X_train = vectorizer.fit_transform(df_train_filtered.text.tolist())
X_test = vectorizer.transform(df_test.text.tolist())

In [49]:
from sklearn.linear_model import LogisticRegression

sklearn_model = LogisticRegression(C=1e3, solver='lbfgs')
sklearn_model.fit(X=X_train, y=preds_train_filtered)

LogisticRegression(C=1000.0)

In [50]:
print(f"Logistic regression accuracy: {sklearn_model.score(X=X_test, y=df_test.label) * 100:.1f}%")

Logistic regression accuracy: 48.5%


As can be seen, the final model improved the score over the majority vote and the `LabelModel` model.

#### assignment

Complete the above calls with functions that you wrote yourself and check whether your functions improve the quality of the model.

## Transforming functions

The idea of a transforming function is to perform an atomic transformation of an instance. For data that is an image, typical transformations include cropping, rotating, and changing the color palette. For text data, you can replace words with synonyms, substitute named entities, cut random pieces of text, etc. In the following example we will find types of named entities occurring in the text, and then prepare a simple transformer that will randomly replace occurrences of the `PERSON` entity

In [51]:
import spacy

nlp = spacy.load('en_core_web_sm')

for doc in nlp.pipe(df_train.text.sample(frac=0.05)):
    print(f"Entities: {[(e.text, e.label_) for e in doc.ents]}")

Entities: []
Entities: [('dis', 'ORG'), ('5', 'CARDINAL')]
Entities: [('RGENT', 'ORG'), ('2nd', 'ORDINAL'), ('U!U', 'WORK_OF_ART'), ('150ppm', 'CARDINAL'), ('50', 'CARDINAL')]
Entities: []
Entities: []
Entities: []
Entities: []
Entities: [('bloo bloo', 'PERSON'), ('first', 'ORDINAL')]
Entities: []
Entities: []
Entities: []
Entities: []
Entities: [('1', 'CARDINAL'), ('08712400200', 'CARDINAL')]
Entities: [('dear.with', 'PERSON'), ('NRI', 'ORG')]
Entities: []
Entities: []
Entities: []
Entities: []
Entities: [('Bin Awarded', 'PERSON'), ('50', 'MONEY'), ('9th', 'ORDINAL'), ('OptOut 08718727870', 'ORG')]
Entities: [('2', 'CARDINAL'), ('0870141701216+', 'DATE'), ('4txt/120p', 'DATE')]
Entities: [('china', 'GPE')]
Entities: []
Entities: [('Sunshine Quiz Wkly', 'PERSON'), ('Sony', 'ORG'), ('Liverpool', 'GPE'), ('mid week', 'DATE'), ('82277', 'DATE'), ('1.50', 'MONEY')]
Entities: [('yahoo', 'ORG')]
Entities: [('s u & de arab', 'ORG'), ('2', 'CARDINAL')]
Entities: [('BRAND', 'ORG'), ('Nokia', 'O

In [52]:
person_entities = []

for doc in nlp.pipe(df_train.text):
    for e in doc.ents:
        if e.label_ == 'PERSON':
            person_entities.append(e.text)
        
person_entities[:10]

['jurong point',
 'Melle Melle',
 'Yummy',
 'Mark',
 'Mallika Sherawat',
 'Divorce Barbie',
 'Ken',
 'Wah',
 'WANNA',
 'Ela']

In [53]:
from snorkel.augmentation import transformation_function
from snorkel.preprocess.nlp import SpacyPreprocessor

spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

@transformation_function(pre=[spacy])
def random_person_ner(sms):
    person_ners = [e.text for e in sms.doc.ents]
    
    if person_ners:
        person_to_replace = np.random.choice(person_ners)
        person_to_add = np.random.choice(person_entities)
        sms.text = sms.text.replace(person_to_replace, person_to_add)
    return sms

Another example of transformation could be using WordNet to find synonyms for words. However, this requires downloading a corpus of data

In [54]:
import nltk
from nltk.corpus import wordnet

nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\erykp\AppData\Roaming\nltk_data...


True

In [55]:
def get_synonym(word):
    
    synsets = wordnet.synsets(word)
    
    if synsets:
        words = [lemma.name() for lemma in synsets[0].lemmas()]
        
        return np.random.choice([w.replace("_", " ") for w in words])


In [56]:
@transformation_function()
def replace_words_with_synonym(sms, num_replacements=5):

    words = sms.text.split()
    
    for _ in range(num_replacements):
        word_idx = np.random.choice(range(len(words)))
        synonym = get_synonym(words[word_idx])
        if synonym:
            words[word_idx] = synonym
        
    sms.text = ' '.join(words)
    return sms

Let us now compare the original text message content with the transformed versions.

In [57]:
# source: https://github.com/snorkel-team/snorkel-tutorials/blob/master/spam/utils.py

from collections import OrderedDict

def preview_tfs(df, tfs):
    transformed_examples = []
    for f in tfs:
        for i, row in df.iterrows():
            transformed_or_none = f(row)
            # If TF returned a transformed example, record it in dict and move to next TF.
            if transformed_or_none is not None:
                transformed_examples.append(
                    OrderedDict(
                        {
                            "TF Name": f.name,
                            "Original Text": row.text,
                            "Transformed Text": transformed_or_none.text,
                        }
                    )
                )
                
    return pd.DataFrame(transformed_examples)


In [58]:
tfs = [random_person_ner, replace_words_with_synonym]

preview_tfs(df_train.sample(frac=0.1), tfs)

,TF Name,Original Text,Transformed Text
0,random_person_ner,What can i do? Might accidant tookplace between somewhere ghodbandar rd. Traffic moves slovely. So plz slip &amp; don't worry.,What can i do? Might accidant tookplace between somewhere ghodbandar rd. Traffic moves slovely. So plz Jamesamp; don't worry.
1,random_person_ner,Sorry. || mail? ||,Sorry. || mail? ||
2,random_person_ner,Wait . I will msg after &lt;#&gt; min.,Wait . I will msg after &lt;#&gt; min.
3,random_person_ner,My sort code is and acc no is . The bank is natwest. Can you reply to confirm i've sent this to the right person!,My sort code is and acc no is . The bank is natwest. Can you reply to confirm i've sent this to the right person!
4,random_person_ner,Then why you came to hostel.,Then why you came to hostel.
...,...,...,...
887,replace_words_with_synonym,Ever thought about living a good life with a perfect partner? Just txt back NAME and AGE to join the mobile community. (100p/SMS),of all time idea about living a good life with a perfect tense partner? just txt back NAME and AGE to join the mobile community. (100p/SMS)
888,replace_words_with_synonym,It is only yesterday true true.,IT be sole yesterday true true.
889,replace_words_with_synonym,"I sent you the prices and do you mean the &lt;#&gt; g,","atomic number 53 sent you the prices and do you mean the &lt;#&gt; g,"
890,replace_words_with_synonym,"A swt thought: ""Nver get tired of doing little things 4 lovable persons.."" Coz..somtimes those little things occupy d biggest part in their Hearts.. Gud ni8","angstrom unit swt thought: ""Nver get tire of doing little things 4 lovable persons.."" Coz..somtimes those little things occupy d biggest part in their Hearts.. Gud ni8"


Applying transforming functions requires some policy defining the order and number of transformations. In the example below, two transformation functions are drawn at random and this sequence of two functions is applied twice to each data point. As a result, we triple the size of the train set.

In [59]:
from snorkel.augmentation import RandomPolicy, PandasTFApplier

random_policy = RandomPolicy(len(tfs), sequence_length=2, n_per_original=2, keep_original=True)

tf_applier = PandasTFApplier(tfs, random_policy)

df_train_sample = df_train.sample(frac=0.1)
df_train_augmented = tf_applier.apply(df_train_sample)

100%|██████████| 446/446 [00:05<00:00, 81.05it/s]


In [60]:
df_train_sample.shape, df_train_augmented.shape

((446, 2), (1338, 2))

#### assignment

Modify the transforming function ``replace_words_with_synonym()`` so that you can restrict the replacement of words with synonyms only for specific parts of speech (e.g., replace only nouns or verbs).